In [47]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Gestion de archivos 🗂️
from pathlib import Path
from tkinter import filedialog
# Coneccion base de datos
from sqlalchemy import create_engine
import cx_Oracle


### Coneccion base de datos Oracle

In [48]:
import configparser

config = configparser.ConfigParser()

config.read('../config.ini')

# Obtener los datos de conexión a la base de datos
host = config['oracle']['host']
port = config['oracle']['port']
sid = config['oracle']['service_name']
user = config['oracle']['user']
password = config['oracle']['password']

# Configuración de la base de datos
dsn = cx_Oracle.makedsn(host, port, sid)

engine = create_engine(f'oracle+cx_oracle://{user}:{password}@{dsn}')

In [49]:
ruta = Path(filedialog.askdirectory())
ruta

WindowsPath('G:/.shortcut-targets-by-id/1buUUJ2naBFTn-E10CXNd8elJ6YQOlWCR/00_BASES_COMPENSACION_2024/Compensación_052024')

In [50]:
# Preguntar periodo de actualizacion
periodo = input("Ingrese el periodo de actualización (YYYYMM): ")
periodo_anterior = str(int(periodo) - 1)
print(f'el periodo a actualizar es {periodo} y el periodo anterior es {periodo_anterior}')

el periodo a actualizar es 202406 y el periodo anterior es 202405


In [51]:
archivos = [archivo for archivo in ruta.iterdir() if archivo.is_file() and archivo.suffix == '.txt']
archivos

[WindowsPath('G:/.shortcut-targets-by-id/1buUUJ2naBFTn-E10CXNd8elJ6YQOlWCR/00_BASES_COMPENSACION_2024/Compensación_052024/AC4023_5_2024.txt'),
 WindowsPath('G:/.shortcut-targets-by-id/1buUUJ2naBFTn-E10CXNd8elJ6YQOlWCR/00_BASES_COMPENSACION_2024/Compensación_052024/CORRECCION_BEN_MAYO_2024.txt'),
 WindowsPath('G:/.shortcut-targets-by-id/1buUUJ2naBFTn-E10CXNd8elJ6YQOlWCR/00_BASES_COMPENSACION_2024/Compensación_052024/CORRECCION_COT_MAYO_2024.txt'),
 WindowsPath('G:/.shortcut-targets-by-id/1buUUJ2naBFTn-E10CXNd8elJ6YQOlWCR/00_BASES_COMPENSACION_2024/Compensación_052024/AB4023_5_2024.txt'),
 WindowsPath('G:/.shortcut-targets-by-id/1buUUJ2naBFTn-E10CXNd8elJ6YQOlWCR/00_BASES_COMPENSACION_2024/Compensación_052024/EPSS05LIQUIDACION01052024.txt'),
 WindowsPath('G:/.shortcut-targets-by-id/1buUUJ2naBFTn-E10CXNd8elJ6YQOlWCR/00_BASES_COMPENSACION_2024/Compensación_052024/EPSS05RESTITUCION01052024.txt')]

In [52]:
beneficiario = ['CORRECCION_BEN', 'AB4023']
cotizante = ['CORRECCION_COT' , 'AC4023']
liquidado = ['EPSS05RESTITUCION', 'EPSS05LIQUIDACION']

## Liquidacion

In [53]:
columnas_liquidados = ['SERIAL_BDUA',
 'TIPO_DOC',
 'NUM_DOC',
 'PRIMER_APELLIDO',
 'SEGUNDO_APELLIDO',
 'PRIMER_NOMBRE',
 'SEGUNDO_NOMBRE',
 'FECHA_NACIMIENTO',
 'GENERO',
 'FECHA_AFILIACION',
 'EPSS',
 'DEPARTAMENTO',
 'MUNICIPIO',
 'TIPO_SUBSIDIO',
 'PERIODO',
 'DIAS',
 'VALOR_UPC',
 'FECHA_PROCESO',
 'ADD_MES_FECHA_NACIMIENTO',
 'EDAD_AFILIADO',
 'GRUPO_ETAREO',
 'ZONA',
 'CENTRO_COSTO']

In [54]:
df_liquidado = []
for archivo in archivos:
    if liquidado[0] in archivo.name or liquidado[1] in archivo.name:
        df = pd.read_csv(archivo, sep=',', dtype='str', encoding='latin1', usecols=columnas_liquidados)
        df_liquidado.append(df)
df_liquidados = pd.concat(df_liquidado)
df_liquidados.head()

,SERIAL_BDUA,TIPO_DOC,NUM_DOC,PRIMER_APELLIDO,SEGUNDO_APELLIDO,PRIMER_NOMBRE,SEGUNDO_NOMBRE,FECHA_NACIMIENTO,GENERO,FECHA_AFILIACION,...,TIPO_SUBSIDIO,PERIODO,DIAS,VALOR_UPC,FECHA_PROCESO,ADD_MES_FECHA_NACIMIENTO,EDAD_AFILIADO,GRUPO_ETAREO,ZONA,CENTRO_COSTO
0,100000416,CC,1006362446,PEREA,PERLAZA,JAIRO,ANDRES,24/03/1997,M,01/02/2024,...,ST,01/05/2024,30,77220.00,01/05/2024,23/04/1997,27,6,GRANDES CIUDADES,1
1,100000679,CC,1107507393,ROJAS,REVELO,TANNIA,YARITZA,08/02/1997,F,02/03/2020,...,ST,01/05/2024,30,122227.80,01/05/2024,07/03/1997,27,7,GRANDES CIUDADES,8
2,100001099,TI,1112153832,JARAMILLO,CERQUERA,CRISTIAN,CAMILO,05/03/2009,M,06/09/2022,...,ST,01/05/2024,30,40267.80,01/05/2024,04/04/2009,15,4,NORMAL,56
3,100001246,CC,11637284,CAICEDO,CAICEDO,LUIS,INOCENSIO,01/02/1972,M,10/08/2021,...,ST,01/05/2024,30,151367.70,01/05/2024,29/02/1972,52,9,ESPECIAL,66
4,100001251,CC,1192923078,GOMEZ,NaN,CRISTIAN,JAVIER,16/01/1998,M,02/09/2023,...,ST,01/05/2024,30,77220.00,01/05/2024,15/02/1998,26,6,GRANDES CIUDADES,2


In [55]:
# Objeto con nombre de tablas
tablas: dict[str, str] = {
    'beneficiario': 'TBL_OPE_COMPENSADO_BEN',
    'cotizante': 'TBL_OPE_COMPENSADO_COT',
    'liquidado': 'EPSS05LIQUIDACION'
}



In [56]:
# Copiar estructura de la tabla EPSS05LIQUIDACION_202405 para crear la tabla EPSS05LIQUIDACION_202406

with engine.connect() as connection:
    
    # Crear cursor
    cursor = connection.connection.cursor()
    
        # Borra tabla si existe
    try:
        cursor.execute(f"""
            DROP TABLE {tablas['liquidado'].lower()}_{periodo}
        """)    
    except Exception as e:
        print(e)
    
    # Crear tabla EPSS05LIQUIDACION_202406
    try:
        cursor.execute(f"""
            CREATE TABLE epss05liquidacion_{periodo} AS
            SELECT * FROM epss05liquidacion_{periodo_anterior} WHERE 1=0
        """)    
    except Exception as e:
        print(e)

# Hacer la insercion por chunk df_liquidados
chunksize = 50000
for i in range(0, df_liquidados.shape[0], chunksize):
    print(i)
    df_liquidados.iloc[i:i+chunksize].to_sql(tablas['liquidado'].lower() + f'_{periodo}', engine, if_exists='append', index=False)

0
50000
100000
150000
200000
250000
300000
350000
400000
450000
500000
550000
600000
650000
700000
750000
800000
850000
900000
950000
1000000
1050000
1100000
1150000
1200000
1250000
1300000
1350000
1400000
1450000
1500000
1550000
1600000


In [57]:
df_liquidados.shape

(1633686, 23)

## Cotizantes

In [58]:
df_cotizantes = []
for archivo in archivos:
    if cotizante[0] in archivo.name or cotizante[1] in archivo.name:
        df = pd.read_csv(archivo, sep=',', dtype='str', encoding='latin1')
        df.rename(columns={'DEPARTAMENTO':'COD_DEP', 'MUNICIPIO':'COD_MUN', 'EXONERACIÓN':'EXONERACION', 'CENTRO_COSTO':'CENTRO_DE_COSTO'}, inplace=True)
        df_cotizantes.append(df)
df_cotizantes = pd.concat(df_cotizantes)
df_cotizantes.head()

,COD_EPS,FECHA_PROCESO,PERIODO_COMPENSADO,TD_COT,NUM_COT,TIPO_COT,TD_NIT,NUM_NIT,IBC_COTIZADO,DIAS_COTIZADOS,...,UPC,PROV_INCAPACIDADES,PYP_EPS,SERIAL_BDUA,SERIAL_HA,COD_DEP,COD_MUN,EXONERACION,CENTRO_DE_COSTO,TIPO_PROCESO
0,EPS005,07/05/2024,05/2024,CC,20774159,10,NI,800224808,1300000,30,...,277249.41,5070.00,2012.70,27442965,1913004393,11,001,NaN,1,SNC
1,EPS005,07/05/2024,04/2024,CC,31529421,1,NI,901160401,1300000,30,...,182391.53,5070.00,2012.70,38639141,1920464247,76,001,S,2,SNC
2,EPS005,07/05/2024,05/2024,CC,7719201,1,NI,800215546,3220004,30,...,76481.22,12558.01,2012.70,27965747,1920855333,11,001,N,1,SNC
3,EPS005,07/05/2024,05/2024,CC,32827536,10,NI,900336004,1300000,30,...,182391.53,5070.00,2012.70,74573622,1923104680,08,001,NaN,4,NORMAL
4,EPS005,07/05/2024,05/2024,CC,51745935,10,NI,900336004,2060623,30,...,196540.10,8036.42,2012.70,39311429,1922571424,15,759,NaN,34,NORMAL


In [59]:
df_cotizantes.columns.to_list()

['COD_EPS',
 'FECHA_PROCESO',
 'PERIODO_COMPENSADO',
 'TD_COT',
 'NUM_COT',
 'TIPO_COT',
 'TD_NIT',
 'NUM_NIT',
 'IBC_COTIZADO',
 'DIAS_COTIZADOS',
 'COTIZACION',
 'F_CONSIGNACION',
 'COD_OPERADOR',
 'NUM_PLANILLA',
 'REGISTRO_COMPENSADO',
 'COD_GLOSA',
 'T_DIAS_COMP',
 'GRUPO_ETAREO',
 'ZONA',
 'UPC',
 'PROV_INCAPACIDADES',
 'PYP_EPS',
 'SERIAL_BDUA',
 'SERIAL_HA',
 'COD_DEP',
 'COD_MUN',
 'EXONERACION',
 'CENTRO_DE_COSTO',
 'TIPO_PROCESO']

In [60]:
# Copiar estructura de la tabla EPSS05LIQUIDACION_202405 para crear la tabla EPSS05LIQUIDACION_202406

with engine.connect() as connection:
    
    # Crear cursor
    cursor = connection.connection.cursor()
    
        # Borra tabla si existe
    try:
        cursor.execute(f"""
            DROP TABLE {tablas['cotizante'].lower()}_{periodo}
        """)    
    except Exception as e:
        print(e)
    
    # Crear tabla EPSS05LIQUIDACION_202406
    try:
        cursor.execute(f"""
            CREATE TABLE {tablas['cotizante'].lower()}_{periodo} AS
            SELECT * FROM {tablas['cotizante'].lower()}_{periodo_anterior} WHERE 1=0
        """)    
    except Exception as e:
        print(e)

# Hacer la insercion por chunk df_liquidados
chunksize = 50000
for i in range(0, df_cotizantes.shape[0], chunksize):
    print(i)
    df_cotizantes.iloc[i:i+chunksize].to_sql(tablas['cotizante'].lower() + f'_{periodo}', engine, if_exists='append', index=False)

0
50000
100000
150000
200000
250000
300000
350000
400000
450000
500000
550000
600000
650000
700000
750000
800000
850000
900000
950000
1000000
1050000
1100000
1150000
1200000
1250000
1300000
1350000
1400000
1450000
1500000
1550000
1600000
1650000
1700000
1750000
1800000
1850000
1900000
1950000
2000000
2050000
2100000
2150000
2200000
2250000
2300000
2350000
2400000
2450000
2500000
2550000
2600000
2650000
2700000
2750000
2800000
2850000
2900000
2950000
3000000
3050000
3100000
3150000
3200000
3250000
3300000
3350000
3400000
3450000
3500000
3550000
3600000
3650000
3700000
3750000
3800000
3850000
3900000
3950000
4000000
4050000
4100000
4150000
4200000
4250000
4300000
4350000
4400000
4450000
4500000
4550000
4600000
4650000
4700000
4750000


## Beneficiarios

In [61]:
df_beneficiario = []
for archivo in archivos:
    if beneficiario[0] in archivo.name or beneficiario[1] in archivo.name:
        df = pd.read_csv(archivo, sep=',', dtype='str', encoding='latin1')
        df.rename(columns={'DEPARTAMENTO':'COD_DEP', 'MUNICIPIO':'COD_MUN', 'EXONERACIÓN':'EXONERACION'}, inplace=True)
        df_beneficiario.append(df)
df_beneficiario = pd.concat(df_beneficiario)
df_beneficiario.head()

,COD_EPS,FECHA_PROCESO,PERIODO_COMPENSADO,TD_COT,NUM_COT,TD_BEN,NUM_BEN,TIPO_AFIL,PARENTESCO,T_DIAS_COMP,...,GRUPO_ETAREO,ZONA,UPC,PYP,SERIAL_BDUA,SERIAL_HA,COD_DEP,COD_MUN,CENTRO_DE_COSTO,TIPO_PROCESO
0,EPS005,17/05/2024,01/2024,CC,1000021137,RC,1024311267,B,0,-1,...,2,3,-3797.84,-67.09,121416448,1860435229,11,001,NaN,NaN
1,EPS005,17/05/2024,01/2024,CC,1000021137,RC,1024311267,B,0,-2,...,2,3,-7595.68,-134.18,121416448,1860435230,11,001,NaN,NaN
2,EPS005,17/05/2024,01/2024,CC,1000021137,RC,1024311267,B,0,-27,...,2,3,-102541.69,-1811.43,121416448,1860435231,11,001,NaN,NaN
3,EPS005,17/05/2024,01/2024,CC,1000159638,RC,1013158683,B,0,-30,...,2,3,-113935.21,-2012.70,118805552,1854212043,11,001,NaN,NaN
4,EPS005,17/05/2024,01/2024,CC,1000729378,RC,1013700440,B,0,-1,...,1,3,-13205.62,-67.09,120711918,1866841837,11,001,NaN,NaN


In [62]:
# Copiar estructura de la tabla EPSS05LIQUIDACION_202405 para crear la tabla EPSS05LIQUIDACION_202406

with engine.connect() as connection:
    
    # Crear cursor
    cursor = connection.connection.cursor()
    
    # Borra tabla si existe
    try:
        cursor.execute(f"""
            DROP TABLE {tablas['beneficiario'].lower()}_{periodo}
        """)    
    except Exception as e:
        print(e)
        
    # Crear tabla TBL_OPE_COMPENSADO_BEN_[PERIODO]
    try:
        cursor.execute(f"""
            CREATE TABLE {tablas['beneficiario'].lower()}_{periodo} AS
            SELECT * FROM {tablas['beneficiario'].lower()}_{periodo_anterior} WHERE 1=0
        """)    
    except Exception as e:
        print(e)

# Hacer la insercion por chunk df_liquidados
chunksize = 50000
for i in range(0, df_beneficiario.shape[0], chunksize):
    print(i)
    df_beneficiario.iloc[i:i+chunksize].to_sql(tablas['beneficiario'].lower() + f'_{periodo}', engine, if_exists='append', index=False)

0
50000
100000
150000
200000
250000
300000
350000
400000
450000
500000
550000
600000
650000
700000
750000
800000
850000
900000
950000
1000000
1050000
1100000
1150000
1200000
1250000
1300000
1350000
1400000
1450000
1500000
1550000
1600000
1650000
1700000
1750000
1800000
1850000
1900000
1950000
2000000
2050000
2100000
2150000
2200000
2250000
2300000
2350000
2400000
2450000
2500000
2550000
